In [1]:
import pandas as pd
from pandas import json_normalize
from datetime import date
pd.set_option('display.max_columns', None)

In [2]:
df_author_raw = catalog.load('raw/openalex/author#parquet')

                    INFO     Loading data from raw/openalex/author#parquet (ParquetDataset)...  ]8;id=466054;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=462660;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

# Profiling

In [3]:
df_author_raw.head(3)

,id,orcid,display_name,display_name_alternatives,works_count,cited_by_count,summary_stats,ids,affiliations,last_known_institutions,topics,topic_share,x_concepts,counts_by_year,works_api_url,updated_date,created_date
0,https://openalex.org/A5003447605,https://orcid.org/0000-0002-5260-1807,Gustavo E. Romero,"[Gustavo Adolfo Sierra Romero, Gustavo Romero,...",766,9617,"{'2yr_mean_citedness': 0.8653846153846154, 'h_...",{'openalex': 'https://openalex.org/A5003447605...,"[{'institution': {'country_code': 'AR', 'displ...","[{'country_code': 'AR', 'display_name': 'Insti...","[{'count': 385, 'display_name': 'Astrophysics ...",[{'display_name': 'Astrophysics and Cosmic Phe...,"[{'display_name': 'Physics', 'id': 'https://op...","[{'cited_by_count': 40, 'works_count': 1, 'yea...",https://api.openalex.org/works?filter=author.i...,2025-01-29T08:43:42.728407,2023-07-21
1,https://openalex.org/A5053874532,None,Guillermo Soberón,"[Guillermo R Soberon, Guillermo Soberon, G. So...",630,1134,"{'2yr_mean_citedness': 0.0, 'h_index': 14, 'i1...",{'openalex': 'https://openalex.org/A5053874532...,"[{'institution': {'country_code': 'MX', 'displ...","[{'country_code': 'MX', 'display_name': 'Mexfa...","[{'count': 146, 'display_name': 'Higher Educat...",[{'display_name': 'Higher Education in Latin A...,"[{'display_name': 'Philosophy', 'id': 'https:/...","[{'cited_by_count': 1, 'works_count': 0, 'year...",https://api.openalex.org/works?filter=author.i...,2025-01-31T02:03:13.610852,2023-07-21
2,https://openalex.org/A5112709549,None,Hugo Luis López,"[Hugo L. López, Hugo Luis López, Hugo L. Lopez...",482,3341,"{'2yr_mean_citedness': 1.3333333333333333, 'h_...",{'openalex': 'https://openalex.org/A5112709549...,"[{'institution': {'country_code': 'BO', 'displ...","[{'country_code': 'BO', 'display_name': 'Oruro...","[{'count': 138, 'display_name': 'Literary and ...",[{'display_name': 'Literary and Cultural Studi...,"[{'display_name': 'Art', 'id': 'https://openal...","[{'cited_by_count': 8, 'works_count': 0, 'year...",https://api.openalex.org/works?filter=author.i...,2025-01-25T07:17:39.319203,2024-10-15


# Nodo

In [4]:
def land_author2topic_openalex(df: pd.DataFrame)-> pd.DataFrame:
    
    df_author = df.loc[:,['id', 'topics']]
    df_author = df_author.convert_dtypes() 
    
    # proceso 'topics'
    df_author2topic_exploded = df_author.explode('topics').reset_index(drop=True)
    
    df_author2topic_norm = pd.json_normalize(df_author2topic_exploded['topics'])
    df_author2topic_norm = df_author2topic_norm.loc[:,['count','id','domain.id','field.id','subfield.id']]
    df_author2topic_norm = df_author2topic_norm.rename(columns={'id':'id_topic'})

    df_author2topic = pd.concat([df_author2topic_exploded, df_author2topic_norm], axis=1)
    df_author2topic = df_author2topic.drop(columns=['topics'])

    df_author2topic.rename(columns={'domain.id':'domain_id'}, inplace=True)
    df_author2topic.rename(columns={'field.id':'field_id'}, inplace=True)
    df_author2topic.rename(columns={'subfield.id':'subfield_id'}, inplace=True)

    df_author2topic['load_datetime'] = date.today()

    return df_author2topic


# Nodo en ejecución

In [5]:
df_author2topic = land_author2topic_openalex(df_author_raw)

# Resultados

In [6]:
df_author2topic.head(3)

,id,count,id_topic,domain_id,field_id,subfield_id,load_datetime
0,https://openalex.org/A5003447605,385,https://openalex.org/T10818,https://openalex.org/domains/3,https://openalex.org/fields/31,https://openalex.org/subfields/3106,2025-02-01
1,https://openalex.org/A5003447605,268,https://openalex.org/T11323,https://openalex.org/domains/3,https://openalex.org/fields/31,https://openalex.org/subfields/3103,2025-02-01
2,https://openalex.org/A5003447605,247,https://openalex.org/T10744,https://openalex.org/domains/3,https://openalex.org/fields/31,https://openalex.org/subfields/3103,2025-02-01
